# Video Description Generation and Query Retrieval

## Overview

This notebook demonstrates how to generate video descriptions using the [**Qwen 2.5 Vision-Language model**](https://github.com/QwenLM/Qwen2.5-VL) and store their embeddings in [**ChromaDB**](https://www.trychroma.com/) for efficient semantic search on **Intel® Core™ Ultra Processors**. The Qwen 2.5 Vision-Language model is loaded using the [**PyTorch XPU backend**](https://docs.pytorch.org/docs/stable/notes/get_start_xpu.html) to leverage Intel hardware acceleration.\
For each video, a description is generated and stored as an embedding in ChromaDB. When a user submits a query, cosine similarity search is performed in ChromaDB to retrieve the most relevant video description. The matching video is then displayed inline.\
This sample supports sports-related queries and uses a subset of videos from the [**Sports Videos in the Wild (SVW)**](https://cvlab.cse.msu.edu/project-svw.html) dataset. For more information on the dataset and citation requirements, please refer to the [**Sports Videos in the Wild (SVW) dataset paper**](https://cvlab.cse.msu.edu/project-svw.html#:~:text=SVW%20Download,Bibtex%20%7C%20PDF).

## Workflow

- During the initial data load, a subset of videos from the [Sports Videos in the Wild (SVW)](https://cvlab.cse.msu.edu/project-svw.html) dataset is fed into the [Qwen 2.5 Vision-Language model](https://github.com/QwenLM/Qwen2.5-VL).
- Here, the [Qwen2.5-VL-3B-Instruct](https://huggingface.co/Qwen/Qwen2.5-VL-3B-Instruct) model variant is used to process these videos and generate descriptions. The Qwen 2.5 Vision-Language model is loaded using the [PyTorch XPU backend](https://docs.pytorch.org/docs/stable/notes/get_start_xpu.html) to leverage Intel hardware acceleration.
- Next, the generated video descriptions are converted into embeddings using [Sentence Transformers](https://sbert.net/), with the [all-MiniLM-L6-v2 model](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2).
- These embeddings, along with the descriptions and video metadata, are stored in a persistent local [ChromaDB](https://www.trychroma.com/) collection. This is a one-time operation; since ChromaDB is local and persistent, it does not need to be repeated unless new videos are added.
- When a user submits a query, the text is similarly encoded into an embedding, which is then used to perform a semantic search (via cosine similarity) over the ChromaDB collection.
- The final result will be the most relevant video description and its associated video file name, and the video is displayed directly in the notebook.

![Video_description_generation_and_query_retrieval_workflow.jpg](./assets/Video_description_generation_and_query_retrieval_workflow.jpg)

## Import necessary packages
Import all the necessary packages and libraries

In [1]:
import os
import torch
import random
import shutil
import zipfile
import logging
import chromadb
import warnings
from tqdm import tqdm
from IPython.display import Video, display
from huggingface_hub import notebook_login
from qwen_vl_utils import process_vision_info
from sentence_transformers import SentenceTransformer
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor

logging.basicConfig(level=logging.INFO)
warnings.filterwarnings("ignore")

## Login to Huggingface to download the models

- Log in to [Huggingface](https://huggingface.co/) using your credentials.
- You’ll need a [User Access Token](https://huggingface.co/docs/hub/security-tokens), which you can generate from your [Settings page](https://huggingface.co/settings/tokens). This token is used to authenticate your identity with the Hugging Face Hub.
- Once you've generated the token, copy it and keep it secure. Then, run the cell below and paste your access token when prompted.

In [2]:
notebook_login()

## Extract ZIP file and get video file paths
Extract the ZIP file which contains the subset of Sports Videos in the Wild (SVW) dataset. Select the number of videos to select in the directory and return the list of file paths of the videos.

In [3]:
def extract_video_paths():
    """
    Extracts video files from the ZIP archive. Select the number of videos to select in the directory and return the video file paths.

    Returns:
        list: Selected list of paths to the extracted video files.

    Raises:
        Exception: Raises an exception if there is any error during extracting video dataset.
    """
    try:
        zip_path = "SVW_subset_video_dataset.zip"
        dataset_folder = "SVW_subset_video_dataset"
        if not os.path.exists(dataset_folder):
            logging.info(" Extracting video dataset..")
            with zipfile.ZipFile(zip_path, "r") as zip_ref:
                zip_ref.extractall()
            logging.info(" Extraction complete.")
        else:
            logging.info(" Found video dataset directory")
        video_extensions = ['.mp4', '.avi', '.mov', '.mkv', '.flv', '.wmv']
        video_files = []
        for root, dirs, files in os.walk(dataset_folder):
            video_files.extend([os.path.join(root, f) for f in files if any(f.lower().endswith(ext) for ext in video_extensions)])
        total_video_files = len(video_files)
        max_videos_to_select = 100
        num_videos_to_select = min(total_video_files, max_videos_to_select)
        random.seed(42)
        selected_video_files = random.sample(video_files, num_videos_to_select)
        logging.info(f" Total number of video files found: {total_video_files}\n")
        logging.info(f" Selected {num_videos_to_select} video files:")
        for file in selected_video_files:
            logging.info(file)
        return selected_video_files
    except Exception as e:
        logging.exception(f" Error while extracting the video paths: {str(e)}")

In [4]:
selected_video_files = extract_video_paths()

INFO:root: Found video dataset directory
INFO:root: Total number of video files found: 100

INFO:root: Selected 100 video files:
INFO:root:SVW_subset_video_dataset\soccer\252___64eaaaffa5284ed8aae4682f4b08e9ba.mp4
INFO:root:SVW_subset_video_dataset\bowling\170___2e2237fd3f1349c6aba27ab497659f27.mp4
INFO:root:SVW_subset_video_dataset\baseball\136___a5ae4b2b86274243b593a3194e2ff6f9.mp4
INFO:root:SVW_subset_video_dataset\weight\15___b3f6ea0174d144b0a4c1fcc7910ec8f5.mp4
INFO:root:SVW_subset_video_dataset\golf\10___b918ec5abe94452795b4f0f65637bd84.mp4
INFO:root:SVW_subset_video_dataset\football\47___9ad16112bc1e4a0c97b066135f03d1b4.mp4
INFO:root:SVW_subset_video_dataset\diving\192___6098c681c7f24973980e51e3b61e4429.mp4
INFO:root:SVW_subset_video_dataset\bowling\60___2417a63dd4b84e02ab8485ec3bf10bb3.mp4
INFO:root:SVW_subset_video_dataset\bmx\329___4ad9a8015a6044468fb06b4ae758eeda.mp4
INFO:root:SVW_subset_video_dataset\tennis\145___14d52737e37b4fbe8b4aadf4fecaa78a.mp4
INFO:root:SVW_subset_vid

## Initialize models
Initialize the Vision Language Model(Qwen2.5-VL-3B-Instruct) and Sentence Transformer model(all-MiniLM-L6-v2).

In [5]:
def initialize_models():
    """
    Initialize Vision Language and Sentence Transformer models.

    Returns:
        dict: Dictionary containg the initialized Vision Language and Sentence Transformer models.

    Raises:
        Exception: Raises an exception if the model loading fails.
    """
    try:
        model_id = "Qwen/Qwen2.5-VL-3B-Instruct"
        embedding_model_id = "all-MiniLM-L6-v2"
        vl_model = Qwen2_5_VLForConditionalGeneration.from_pretrained(pretrained_model_name_or_path=model_id,
                                                                      torch_dtype=torch.float16)
        processor = AutoProcessor.from_pretrained(pretrained_model_name_or_path=model_id,
                                                  min_pixels=256*14*14,
                                                  max_pixels=768*28*28,
                                                  use_fast=True)
        device = "xpu" if torch.xpu.is_available() else "cpu"
        vl_model = vl_model.to(device)
        logging.info(f" Loading Qwen Vision Language Model: {model_id}")
        logging.info(f" Loading Qwen Vision Language Model onto the Pytorch device: {device}")
        embedding_model = SentenceTransformer(embedding_model_id)        
        return {
            "vl_model": vl_model,
            "processor": processor,
            "embedding_model": embedding_model
        }
    except Exception as e:
        logging.exception(f" Error while loading the models: {str(e)}")

In [6]:
models = initialize_models()
vl_model = models["vl_model"]
processor = models["processor"]
embedding_model = models["embedding_model"]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:root: Loading Qwen Vision Language Model: Qwen/Qwen2.5-VL-3B-Instruct
INFO:root: Loading Qwen Vision Language Model onto the Pytorch device: xpu
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


## Get or create Chromadb collection
Connects to the persistent ChromaDB client if already created. If there is no collection available it is going to create a new persistant ChromaDB collection for storing the Video descriptions and filenames. Here we are using cosine similarity for space parameter which will define the distance function of embedding space.

In [7]:
def get_or_create_database():
    """
    Connects to or creates a persistant ChromaDB collection for storing the Video descriptions and filenames.

    Returns:
        collection: The ChromaDB collection object.
        existing_descriptions (dict): The dictionary contains video descriptions of the existing Chromadb database collection.

    Raises:
        Exception: Raises an exception if there is any error while checking an existing database.
    """
    try:
        client = chromadb.PersistentClient(path="Video_descriptions_database")
        collection = client.get_or_create_collection(
            name="Video_descriptions", 
            configuration={
                "hnsw": {
                    "space": "cosine",
                },
            }
        )
        logging.info(" Checking existing descriptions in database..")
        all_items = collection.get(include=["metadatas", "documents"])
        existing_descriptions = {}
        for metadata, doc in zip(all_items['metadatas'], all_items['documents']):
            existing_descriptions[metadata['video_filename']] = doc
        logging.info(f" Found {len(existing_descriptions)} existing descriptions")
        return collection, existing_descriptions
    except Exception as e:
        existing_descriptions = {}
        logging.exception(" No existing descriptions found")
        logging.exception(f" Error while checking an existing database: {str(e)}")

In [8]:
collection, existing_descriptions = get_or_create_database()

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:root: Checking existing descriptions in database..
INFO:root: Found 0 existing descriptions


## Generate and store video descriptions
Each file in the selected video files list will be checked to determine whether it has already generated video description and stored in the database. If the file is not present in the collection database, a description will be generated by the Qwen model and stored in the collection database.

In [9]:
def generate_and_store_video_descriptions(selected_video_files, collection, existing_descriptions, vl_model, processor, embedding_model):
    """
    Check and generate video description for each video file in the selected directory.
    Store the embeddings of generated video descriptions in the ChromaDB collection.
    
    Args:
        selected_video_files (list): List of selected video files.
        collection: The ChromaDB collection object.
        existing_descriptions (dict): The dictionary contains video descriptions of the existing Chromadb database collection.
        vl_model: Vision Language Model for generating video descriptions.
        processor: Vision Language Model processor for processing videos.
        embedding_model: Sentence Transformer Model for generating embeddings.

    Raises:
        Exception: Raises an exception if there is any error while generating video descriptions or storing the embeddings in the database.
    """
    try:
        video_descriptions = {}
        for i, video_file in enumerate(tqdm(selected_video_files)):
            video_filename = os.path.basename(video_file)
            if video_filename in existing_descriptions:
                logging.info(f" Skipping file {os.path.basename(video_file)} - video description already present in the database")
                video_descriptions[video_file] = existing_descriptions[video_filename]
                continue

            video_path = video_file
            torch.xpu.empty_cache()
            logging.info(f"\n Processing file {os.path.basename(video_file)} using Qwen 2.5 VL on Pytorch XPU..")
            messages = [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "video",
                            "video": video_path,
                            "max_pixels": 320*240,
                            "fps": 0.5,
                        },
                        {
                            "type": "text",
                            "text": "Describe this sports video."
                        },
                    ],
                }
            ]
            text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            image_inputs, video_inputs = process_vision_info(messages)
            inputs = processor(
                text=[text],
                images=image_inputs,
                videos=video_inputs,
                padding=True,
                return_tensors="pt",
            )
            inputs = inputs.to("xpu")
            generated_ids = vl_model.generate(**inputs, max_new_tokens=128)
            generated_ids_trimmed = [
                out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
            ]
            output_text = processor.batch_decode(
                generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
            )
            description_text = output_text[0] if isinstance(output_text, list) else str(output_text)
            video_descriptions[video_file] = description_text
            logging.info(f" Generated description: {description_text}.\n")
            # video_descriptions[video_file] = output_text
            embedding = embedding_model.encode(sentences=description_text).tolist()
            collection.add(
                embeddings=[embedding],
                documents=[description_text],
                metadatas=[{"video_filename": os.path.basename(video_file)}],
                ids=[video_file]
            )
            logging.info(f" Added {os.path.basename(video_file)} video file description to database\n\n ******")
        logging.info(f" Processed {len(video_descriptions)} videos")
        logging.info(f" Database now has {collection.count()} total descriptions")
    except Exception as e:
        logging.exception(f" Error while generating and storing video descriptions: {str(e)}")

<div class="alert alert-warning" role="alert">
  Generating video descriptions and updating the database. This may take some time.
</div>

In [10]:
generate_and_store_video_descriptions(selected_video_files, collection, existing_descriptions, vl_model, processor, embedding_model)    

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]INFO:root:
 Processing file 252___64eaaaffa5284ed8aae4682f4b08e9ba.mp4 using Qwen 2.5 VL on Pytorch XPU..
qwen-vl-utils using decord to read video.
INFO:qwen_vl_utils.vision_process:decord:  video_path='SVW_subset_video_dataset\\soccer\\252___64eaaaffa5284ed8aae4682f4b08e9ba.mp4', total_frames=193, video_fps=30.0, time=0.037s
INFO:root: Generated description: The video shows a group of people playing soccer on an outdoor field. The players are wearing red jerseys and are actively engaged in the game, with one player kicking the ball towards the goal. The background features a cityscape with buildings and hills, suggesting that the field is located in an urban area. The sky is clear and sunny, indicating good weather conditions for playing soccer..



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root: Added 252___64eaaaffa5284ed8aae4682f4b08e9ba.mp4 video file description to database

 ******
  1%|▊                                                                               | 1/100 [01:03<1:44:28, 63.32s/it]INFO:root:
 Processing file 170___2e2237fd3f1349c6aba27ab497659f27.mp4 using Qwen 2.5 VL on Pytorch XPU..
INFO:qwen_vl_utils.vision_process:decord:  video_path='SVW_subset_video_dataset\\bowling\\170___2e2237fd3f1349c6aba27ab497659f27.mp4', total_frames=113, video_fps=30.0, time=0.004s
INFO:root: Generated description: The video shows a man bowling in a bowling alley. He is wearing a brown shirt and khaki pants, and he is standing on the lane with his feet shoulder-width apart. He swings his arm back and then throws the ball down the lane. The ball rolls down the lane and hits the pins at the end of the lane. The man then walks back to his ball return and picks up another ball. He repeats the process of swinging his arm back and throwing the ball down the lane..



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root: Added 170___2e2237fd3f1349c6aba27ab497659f27.mp4 video file description to database

 ******
  2%|█▌                                                                              | 2/100 [02:15<1:51:59, 68.57s/it]INFO:root:
 Processing file 136___a5ae4b2b86274243b593a3194e2ff6f9.mp4 using Qwen 2.5 VL on Pytorch XPU..
INFO:qwen_vl_utils.vision_process:decord:  video_path='SVW_subset_video_dataset\\baseball\\136___a5ae4b2b86274243b593a3194e2ff6f9.mp4', total_frames=87, video_fps=30.0, time=0.000s
INFO:root: Generated description: The video captures a baseball game in progress. The pitcher is on the mound, preparing to throw the ball. The batter is at home plate, ready to swing. The catcher and umpire are behind the batter, ready to react to the pitch. The field is well-maintained, with green grass and a dirt infield. The sky is cloudy, suggesting it might be a cool day. The players are dressed in their uniforms, indicating they are professional or semi-professional. The video i

...

INFO:root: Added 2___59589caa29034f41b53e0aa19227a519.mp4 video file description to database

 ******
 98%|█████████████████████████████████████████████████████████████████████████████▍ | 98/100 [1:53:11<02:20, 70.09s/it]INFO:root:
 Processing file 58___bed1133eca984767a67a1e8e68160a47.mp4 using Qwen 2.5 VL on Pytorch XPU..
INFO:qwen_vl_utils.vision_process:decord:  video_path='SVW_subset_video_dataset\\gymnastics\\58___bed1133eca984767a67a1e8e68160a47.mp4', total_frames=218, video_fps=30.0, time=0.016s
INFO:root: Generated description: The video shows an indoor trampoline park with various trampolines and equipment. A young boy is seen jumping on one of the trampolines, performing flips and tricks. The video captures the excitement and energy of the trampoline park, with other people visible in the background..



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root: Added 58___bed1133eca984767a67a1e8e68160a47.mp4 video file description to database

 ******
 99%|██████████████████████████████████████████████████████████████████████████████▏| 99/100 [1:53:53<01:01, 61.79s/it]INFO:root:
 Processing file 27___eedf5982ac514735a209e9b349bce298.mp4 using Qwen 2.5 VL on Pytorch XPU..
INFO:qwen_vl_utils.vision_process:decord:  video_path='SVW_subset_video_dataset\\hammerthrow\\27___eedf5982ac514735a209e9b349bce298.mp4', total_frames=311, video_fps=30.0, time=0.014s
INFO:root: Generated description: A shirtless man stands in a throwing cage, preparing to throw a discus. He takes a few steps back and begins to spin, building momentum. As he spins, the discus is visible in his hand, held out in front of him. He continues to spin, gaining speed, until he releases the discus with a powerful throw. The discus flies through the air, leaving a trail of dust behind it..



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root: Added 27___eedf5982ac514735a209e9b349bce298.mp4 video file description to database

 ******
100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [1:54:57<00:00, 68.97s/it]
INFO:root: Processed 100 videos
INFO:root: Database now has 100 total descriptions


## Query the database
The query will be encoded, converted into embedding which will be used to query the ChromaDB collection to retrieve videos that are most relevant by searching for similar content based on the cosine similarity.

In [11]:
def query_videos_descriptions(query, collection):
    """
    Queries the ChromaDB collection to find the similar video relevant to the input query.

    Args:
        query (str): User query.
        collection: The ChromaDB collection object.

    Returns:
        results (dict): Query results containing video descriptions and video file name.

    Raises:
        Exception: Raises an exception if there is any error while querying the database.
    """
    try:
        query_embedding = embedding_model.encode(query).tolist()
        results = collection.query(
            query_embeddings=[query_embedding],
            n_results=1,
            include=["documents", "metadatas", "distances", "embeddings"]
        )
        logging.info(f" Search results for: '{query}'\n")
        for i, (doc, metadata, distance) in enumerate(zip(
            results['documents'][0],
            results['metadatas'][0],
            results['distances'][0]
        )):
            similarity_score = 1 - distance
            logging.info(f" Video filename: {metadata['video_filename']} (Similarity score: {similarity_score:.3f})\n")
            logging.info(f" Distance: {distance:.3f}\n")
            logging.info(f" Video description: {doc}\n")
        return results
    except Exception as e:
        logging.exception(f" Error while querying video descriptions: {str(e)}")

In [12]:
query = "girl practicing in a golf club"
results = query_videos_descriptions(query, collection)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root: Search results for: 'girl practicing in a golf club'

INFO:root: Video filename: 10___b918ec5abe94452795b4f0f65637bd84.mp4 (Similarity score: 0.720)

INFO:root: Distance: 0.280

INFO:root: Video description: The video shows a young girl practicing her golf swing on a grassy course. She is holding a golf club and appears to be hitting golf balls. The background features other people, likely fellow golfers or spectators, and the landscape includes mountains in the distance. The setting suggests a sunny day with clear skies.



## Display the video
Display the relevant video based on the results

In [13]:
def display_video(results):
    """
    Display the video based the query results.

    Args:
        results (dict): Query results containing video descriptions and video file name.

    Raises:
        Exception: Raises an exception if there is any error while displaying the video.
    """
    try:
        video = Video(results['ids'][0][0], width=600, height=400)
        display(video)
    except Exception as e:
        logging.exception(f"Error while displaying the video: {str(e)}")

In [14]:
display_video(results)

## Remove the database
Remove the local database if it is not needed.

In [19]:
def delete_database():
    """
    Delete the database directory.

    Raises:
        Exception: Raises an exception if there is any error while deleting the database.
    """
    database_folder = "Video_descriptions_database"
    if os.path.exists(database_folder):
        logging.info("You are about to delete the database. Once deleted, it will no longer be available, and you will need to regenerate and store the video descriptions again.")
        answer = input(f"Do you want to remove the database '{database_folder}'? \nType 'yes'/ 'no': ")
        if answer == 'yes':
            try:
                shutil.rmtree(database_folder)
                logging.info("Database deleted!")
            except Exception as e:
                logging.exception(f"Error while deleting the database: {str(e)}")
        else:
            logging.info("Database not deleted")
    else:
        logging.info("Database is not available")

In [20]:
delete_database()

INFO:root:You are about to delete the database. Once deleted, it will no longer be available, and you will need to regenerate and store the video descriptions again.


Do you want to remove the database 'Video_descriptions_database'? 
Type 'yes'/ 'no':  no


INFO:root:Database not deleted


## Citations

If you use SVW dataset, please refer to this paper in your publications:

### Publications
- [**Sports Videos in the Wild (SVW): A Video Dataset for Sports Analysis**](https://cvlab.cse.msu.edu/project-svw.html)\
[**Seyed Morteza Safdarnejad**](https://cvlab.cse.msu.edu/author/seyed-morteza-safdarnejad.html), [**Xiaoming Liu**](https://cvlab.cse.msu.edu/author/xiaoming-liu.html), [**Lalita Udpa**](https://cvlab.cse.msu.edu/author/lalita-udpa.html), [**Brooks Andrus**](https://cvlab.cse.msu.edu/author/brooks-andrus.html), [**John Wood**](https://cvlab.cse.msu.edu/author/john-wood.html), [**Dean Craven**](https://cvlab.cse.msu.edu/author/dean-craven.html)\
Proc. International Conference on Automatic Face and Gesture Recognition (FG 2015), Ljubljana, Slovenia, May. 2015 (Acceptance rate 84/221 = 38%)\
[**Bibtex**](https://cvlab.cse.msu.edu/project-svw.html#bibtex-sports-videos-in-the-wild-svw-a-video-dataset-for-sports-analysis) | [**PDF**](https://cvlab.cse.msu.edu/pdfs/Morteza_FG2015.pdf)